## Optimizing E-commerce Last-Mile Deliveries using Python

Organizing your routes to deliver parcels with min drivers using optimization models with Python

### Intro to the case study

### Data consists: 

#### The operational manager in a local service centre requested our support to manage his fleet

1. Four drivers in team
2. 15 parcel capacity per vehicle
3. 16 destinations to cover in the neighborhood named Dj with j in [1, 16]
4. D0 is the center point
5. One route per driver


### Using capacitated vehicle routing problem (CVRP) with Google OR-Tools.

#### OR-Tools is an open-source collection of Google with tools for combinatorial optimization.

In [3]:
#importing Libraries

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd
import numpy as np

### Building Model using distance matrix as input

In [5]:
#importing distance matrix 

df_distance = pd.read_excel('df_distance_matrix.xlsx',index_col = 0)

In [6]:
df_distance.head()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16
row0,0,548,776,696,582,274,502,194,308,194,536,502,388,354,468,776,662
row1,548,0,684,308,194,502,730,354,696,742,1084,594,480,674,1016,868,1210
row2,776,684,0,992,878,502,274,810,468,742,400,1278,1164,1130,788,1552,754
row3,696,308,992,0,114,650,878,502,844,890,1232,514,628,822,1164,560,1358
row4,582,194,878,114,0,536,764,388,730,776,1118,400,514,708,1050,674,1244


In [7]:
distance_matrix = df_distance.to_numpy()

### Demand: number of parcels to deliver to each location
### Using a Python list with the first value at zero (because we don’t need anything at the center)

#### Demand = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]

### Objective: Deliver all parcels with a minimum number of drivers & Optimize the routing to minimize the distance covered per route

In [8]:
#Creating dictionnary with data

data = {}
data['distance_matrix'] = distance_matrix
print("{:,} destinations".format(len(data['distance_matrix'][0]) - 1))

# Orders quantity (Boxes)
data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
# Vehicles Capacities (Boxes)
data['vehicle_capacities'] = [15, 15, 15, 15]
# Fleet informations
# Number of vehicles
data['num_vehicles'] = 4
# Location of the depot
data['depot'] = 0

16 destinations


In [9]:
# Create Dataframe of Matrix Distance
def create_excel(data):
    n_col = len(data['distance_matrix'][0])
    n_row = len(data['distance_matrix'])
    list_row = ['row' + str(i) for i in range(n_row)]
    list_col = ['col' + str(i) for i in range(n_row)]

    matrix = np.array(data['distance_matrix'])
    df = pd.DataFrame(data=matrix, index=list_row, columns=list_col)
    df.to_excel('df_distance_matrix.xlsx')

In [10]:
# Orders quantity (Boxes)
data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
# Vehicles Capacities (Boxes)
data['vehicle_capacities'] = [15, 15, 15, 15]
# Fleet informations
# Number of vehicles
data['num_vehicles'] = 4
# Location of the depot
data['depot'] = 0

## Calculate Distances

In [11]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

In [12]:
def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

In [13]:
# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])

# Create Routing Model
routing = pywrapcp.RoutingModel(manager)

# Create and register a transit callback.
transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Capacity constraint.
demand_callback_index = routing.RegisterUnaryTransitCallback(
    demand_callback)
routing.AddDimensionWithVehicleCapacity(demand_callback_index,
    0,  # null capacity slack
    data['vehicle_capacities'],  # vehicle maximum capacities
    True,  # start cumul to zero
    'Capacity')

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.FromSeconds(1)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

In [14]:

if solution:
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for driver {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Parcels({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Parcels({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {} (m)\n'.format(route_distance)
        plan_output += 'Parcels Delivered: {} (parcels)\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {:,} (m)'.format(total_distance))
    print('Parcels Delivered: {:,}/{:,}'.format(total_load, sum(data['demands'])))
else:
    print('No Solution')

Route for driver 0:
 0 Parcels(0) ->  4 Parcels(4) ->  3 Parcels(6) ->  1 Parcels(7) ->  7 Parcels(15) ->  0 Parcels(15)
Distance of the route: 1552 (m)
Parcels Delivered: 15 (parcels)

Route for driver 1:
 0 Parcels(0) ->  14 Parcels(4) ->  16 Parcels(12) ->  10 Parcels(14) ->  9 Parcels(15) ->  0 Parcels(15)
Distance of the route: 1552 (m)
Parcels Delivered: 15 (parcels)

Route for driver 2:
 0 Parcels(0) ->  12 Parcels(2) ->  11 Parcels(3) ->  15 Parcels(11) ->  13 Parcels(15) ->  0 Parcels(15)
Distance of the route: 1552 (m)
Parcels Delivered: 15 (parcels)

Route for driver 3:
 0 Parcels(0) ->  8 Parcels(8) ->  2 Parcels(9) ->  6 Parcels(13) ->  5 Parcels(15) ->  0 Parcels(15)
Distance of the route: 1552 (m)
Parcels Delivered: 15 (parcels)

Total distance of all routes: 6,208 (m)
Parcels Delivered: 60/60


In [19]:
import folium

# Example coordinates (latitude, longitude) for visualization
# Replace with actual depot and location coordinates.
locations = [
    (40.7128, -74.0060),  # Depot
    (40.7306, -73.9352),  # Customer 1
    (40.6500, -73.9498),  # Customer 2
    (40.7000, -73.8000),  # Customer 3
]

# Initialize a Folium map
route_map = folium.Map(location=locations[0], zoom_start=12)

# Plot depot
folium.Marker(locations[0], popup="Depot", icon=folium.Icon(color="red")).add_to(route_map)

# Plot customer locations
for i, loc in enumerate(locations[1:], start=1):
    folium.Marker(loc, popup=f"Customer {i}").add_to(route_map)

# Draw routes for each vehicle
# Example: Replace `routes` with the actual routes from your solution.
routes = [[0, 1, 2, 0], [0, 3, 0]]  # Example routes

colors = ['blue', 'green', 'purple', 'orange']  # Colors for each vehicle
for vehicle_id, route in enumerate(routes):
    route_coordinates = [locations[node] for node in route]
    folium.PolyLine(route_coordinates, color=colors[vehicle_id], weight=2.5, opacity=1).add_to(route_map)

# Save and display the map
route_map.save("route_map.html")


In [20]:
# Example data (replace with actual data from your solution)
report_data = {
    'Vehicle': [1, 2, 3, 4],
    'Total Distance (m)': [1200, 1500, 1100, 900],
    'Total Load Delivered': [10, 12, 8, 7],
    'Idle Time (minutes)': [5, 7, 3, 6],
}

# Create a DataFrame
df_report = pd.DataFrame(report_data)

df_report

,Vehicle,Total Distance (m),Total Load Delivered,Idle Time (minutes)
0,1,1200,10,5
1,2,1500,12,7
2,3,1100,8,3
3,4,900,7,6


In [25]:
from fpdf import FPDF

# Initialize PDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()
pdf.set_font("Arial", size=12)

# Add Title
pdf.set_font("Arial", style="B", size=16)
pdf.cell(200, 10, txt="Vehicle Routing Detailed Report", ln=True, align="C")

# Add Table Headers
pdf.set_font("Arial", size=12)
headers = ["Vehicle", "Total Distance (m)", "Total Load Delivered", "Idle Time (minutes)"]
for header in headers:
    pdf.cell(50, 10, txt=header, border=1)
pdf.ln()

# Add Table Rows
for _, row in df_report.iterrows():
    for item in row:
        pdf.cell(50, 10, txt=str(item), border=1)
    pdf.ln()

# Save PDF
pdf.output("detailed_report.pdf")

print("PDF report saved as 'detailed_report.pdf'")

PDF report saved as 'detailed_report.pdf'
